# Notebook 1 – Hello, Light Theory Realm

**Level:** Beginner  
**Topics:** QGT, Fisher metric, Berry curvature, Standard Model masses, Pocket_U Lite

> **Provenance:** This notebook is based on:
> - `GETTING_STARTED.md`
> - `docs/QUICKSTART.md`
> - `examples/pocket_u_mass_table.py`

## 1. Setup

In this section you'll:

- Install `light-theory-realm` (if needed)
- Import the main public API
- Check the installed version

In [ ]:
# If you're running from a fresh environment (not the repo), uncomment this:
# !pip install light-theory-realm

In [ ]:
import jax
import jax.numpy as jnp

from light_theory_realm import (
    CliffordEngine,
    CliffordQGT,
    print_sm_table,
    get_particle_profile,
)

# Prefer top-level export if it ever exists, but fall back to the subpackage
try:
    from light_theory_realm import koide_ratio  # future-friendly
except ImportError:
    try:
        from light_theory_realm.pocket_u_lite import koide_ratio
        print("Imported koide_ratio from light_theory_realm.pocket_u_lite")
    except ImportError:
        koide_ratio = None
        print(
            "Warning: koide_ratio not found in light_theory_realm or "
            "light_theory_realm.pocket_u_lite. Koide teaser will be skipped."
        )

import light_theory_realm as ltr

print("Light Theory Realm version:", getattr(ltr, "__version__", "unknown"))
print("JAX version:", jax.__version__)
print("koide_ratio available:", koide_ratio is not None)

## 2. Your First QGT: Fisher + Berry in One Line

The Quantum Geometric Tensor (QGT) splits into:

- **Fisher metric** – how fast a state changes as you vary parameters (information distance)
- **Berry curvature** – how much geometric *twist* or topological phase it accumulates

We'll measure this for a random state and a random "direction of change".

In [ ]:
# Initialize engine and QGT geometry object
engine = CliffordEngine(seed=42)
qgt = CliffordQGT(engine)

# Create a state and its derivative
key_psi, key_dpsi = jax.random.split(jax.random.PRNGKey(0))
psi = engine.random_spinor(key_psi)
d_psi = engine.random_spinor(key_dpsi)

# Compute geometry: Fisher (metric) + Berry (curvature)
fisher, berry = qgt.compute_full_qgt(psi, d_psi.reshape(-1, 1))

print("Fisher metric shape:", fisher.shape)
print("Berry curvature shape:", berry.shape)
print()
print(f"Fisher g_00 (information distance): {float(fisher[0, 0]):.6f}")
print(f"Berry Ω_00 (topological twist):     {float(berry[0, 0]):.6f}")

**Interpretation**

- A **larger Fisher component** means the state is more sensitive to small parameter changes.
- A **non-zero Berry component** signals geometric / topological structure – the state picks up a phase as you move around in parameter space.

## 3. Standard Model Snapshot (Pocket_U Lite)

The easiest way to use Light Theory Realm:

- Call `print_sm_table()` to show all 9 fermion masses and errors.
- Call `get_particle_profile("e")` to zoom in on the electron.

This sits at **Layer 4: Experiments** in the 4-layer stack.

In [ ]:
print_sm_table()

> Note: This notebook includes a small robustness patch for `compute_reeb_diagnostics` (safe to re-run; idempotent).


In [ ]:
import light_theory_realm.pocket_u_lite.geometry as geom

# Only save the original once, even if you re-run this cell
if not hasattr(geom, "_orig_compute_reeb_diagnostics"):
    geom._orig_compute_reeb_diagnostics = geom.compute_reeb_diagnostics

def compute_reeb_diagnostics_fixed(primes, *args, **kwargs):
    """
    Wrapper around compute_reeb_diagnostics that ensures `primes` is a tuple,
    so expressions like `primes + (primes[0],)` work even if `primes` was a list.
    """
    primes = tuple(primes)  # list -> tuple (tuple -> tuple is also fine)
    return geom._orig_compute_reeb_diagnostics(primes, *args, **kwargs)

# Monkeypatch the module-level function used by get_particle_geometry
geom.compute_reeb_diagnostics = compute_reeb_diagnostics_fixed

print("Patched pocket_u_lite.geometry.compute_reeb_diagnostics (idempotent).")

In [ ]:
e = get_particle_profile("e")

print("Electron profile:")
print(f"  Physical mass m_phys_MeV       : {e['m_phys_MeV']:.6f} MeV")
print(f"  PDG reference mass m_ref_MeV   : {e['pdg_mass_MeV']:.6f} MeV")
print(f"  Error vs PDG                   : {e['error_pct']:.3f}%")
print(f"  Screening (vacuum bubble)      : {e['screening_pct']:.2f}%")
print(f"  Dark energy density ξ          : {e['xi']:.6f}")

**What you're seeing**

- The **Pocket_U Lite** model predicts fermion masses from prime plaquettes.
- The electron's **screening** is close to 100%, which you can think of as a "vacuum bubble" wrapped around the mass.
- The **ξ parameter** measures a kind of dark-energy–like resonance density in the Reeb flow picture.

## 4. Koide Teaser: A 45° Angle Between Masses

The Koide relation between the three charged leptons `(e, μ, τ)` is famously close to:

$$
Q \approx \frac{2}{3}
$$

We'll just evaluate the Koide ratio here as a teaser for Notebook 5.

In [ ]:
if koide_ratio is None:
    print(
        "koide_ratio is not available in this environment.\n"
        "Upgrade / reinstall light_theory_realm or use the CLI/koide_analysis "
        "example for Koide explorations.\n"
        "Skipping Koide teaser for now."
    )
else:
    # PDG-like lepton masses in MeV
    m_e = 0.511
    m_mu = 105.66
    m_tau = 1776.86

    Q = koide_ratio(m_e, m_mu, m_tau)
    print(f"Koide ratio Q(e, μ, τ) = {Q:.6f} (target ≈ 0.666667)")

## 5. Where to Go Next

You've now:

- Computed your **first QGT** (Fisher + Berry)
- Peeked at **Standard Model mass predictions**
- Seen a teaser of the **Koide relation**

**Recommended next notebooks:**

- 👉 **Notebook 2 – Clifford Algebra 101**  
  Learn how wedge products and grade projections build areas, volumes, and 4D volumes.

- 👉 **Notebook 3 – Basic QGT (Single Parameter)**  
  Follow a 1-parameter curve in Hilbert space and plot how Fisher and Berry change with the angle θ.